In [7]:
import aocd
data = aocd.get_data(day=1, year=2023)
T = data.split()

trans = {'one':'o1e', 'two':'t2o', 'three':'3e', 'four':'4',
         'five':'5e', 'six':'6', 'seven':'7n', 'eight':'e8t',
         'nine':'n9e'}

def tr(x):
    for s in trans: x = x.replace(s, trans[s])
    return x

def cal(x):
    l = [int(c) for c in x if c.isdigit()]
    return 10*l[0]+l[-1]

print("Part1:", sum(map(cal, T)), "Part2:",sum(map(cal, map(tr,T))))

Part1: 54388 Part2: 53515


In [23]:
import aocd
from collections import defaultdict
data = aocd.get_data(day=2, year=2023)

part1 = part2 = 0

for i, g in enumerate(data.split("\n")):
    _,l = g.split(": ")
    p1 = i+1
    maxes = defaultdict(int)
    for u in l.split("; "):
        for t in u.split(", "):
            n, c = t.split(" ")
            n = int(n)
            if not((c == "red" and n <= 12) 
                or (c == "green" and n <= 13) 
                or (c == "blue" and n <= 14)): p1 = 0
            maxes[c] = max(maxes[c], n)
    part1 += p1
    p2 = 1
    for k in maxes: p2 *= maxes[k]
    part2 += p2
print(part1,part2)

2795 75561


In [54]:
import aocd
data = aocd.get_data(day=3, year=2023)

mp = defaultdict(str)
T = data.split('\n')
n = len(T)
m = len(T[0])

for i,l in enumerate(T):
    for j,c in enumerate(l):
        if c != '.': 
            mp[(i,j)] = c

part1 = part2 = 0
gears = defaultdict(list)
tmp, ispart, cgear = 0, False, set()

for i in range(n):
    for j in range(m):
        if mp[(i,j)].isdigit():
            tmp = tmp*10+int(mp[(i,j)])
            for k in range(-1,2):
                for l in range(-1,2):
                    if mp[(i+k,j+l)] != '' and not(mp[(i+k,j+l)].isdigit()):
                        ispart = True
                    if mp[(i+k,j+l)] == '*': cgear.add((i+k,j+l))           
        else:
            if ispart:
                part1 += tmp
                for g in cgear: gears[g].append(tmp)
            tmp, ispart, cgear = 0, False, set()
    if ispart: 
        part1 += tmp
        for g in cgear: gears[g].append(tmp)
    tmp, ispart, cgear = 0, False, set()

for g in gears:
    if len(gears[g]) == 2:
        part2 += gears[g][0]*gears[g][1]
print("part1:",part1,"part2:",part2)

part1: 535351 part2: 87287096


In [32]:
import aocd
data = aocd.get_data(day=4, year=2023).splitlines()

part1, part2, n, cards = 0, 0, len(data), [1]*len(data)

for i, row in enumerate(data):
    win, hand = row.split(':')[1].split("|")
    match = len(set(win.split()).intersection(hand.split()))
    if match >= 1: part1 += 2**(match-1)
    for j in range(i+1, min(i+1+match, n)): cards[j] += cards[i]
    part2 += cards[i]

print("part1:", part1, "part2:", part2)

part1: 18653 part2: 5921508


In [46]:
import aocd
data = aocd.get_data(day=5, year=2023).split("\n\n")

seeds = list(map(int,data[0].split()[1:]))
part1 = [(seed,seed+1) for seed in seeds]
part2 = [(seeds[2*i],seeds[2*i]+seeds[2*i+1])
         for i in range(len(seeds)//2)]

def desc2trans(desc): return [ (src,src+length,dest-src)
           for line in desc.split('\n')[1:]
           for dest,src,length in [map(int,line.split())]]

def step(s2,transitions):
    ret = []
    for sta,sto in s2:
        for start, stop, delta in transitions:
            if start <= sta < stop:
                ret.append((sta+delta,min(sto,stop)+delta))
                sta, sto = min(sto,stop), sto
        if sta<sto: ret.append((sta,sto))
    return ret

for desc in data[1:]:
    transitions = sorted(desc2trans(desc))
    part1 = step(part1,transitions)
    part2 = step(part2,transitions)

print("part1:",min(part1)[0],"part2:",min(part2)[0])

part1: 199602917 part2: 2254686


In [108]:
import aocd, math, time

data = aocd.get_data(day=6, year=2023).split('\n')
races = list(zip(*map(lambda line: list(map(int,line.split()[1:])), data)))
t,d = map(lambda line: int(line.split(":")[1].replace(" ","")), data)

def win(t,d): return(t-1-int((t-math.sqrt(t*t -4*d))/2)*2)

def win_bf(t,d): return sum(1 for ti in range(t) if ti*(t-ti)>d )

print("Root calculation")
start_time = time.time()
print("part1:", math.prod(win(t,d) for t,d in races), "part2:", win(t,d))
print("--- %s seconds ---" % ((time.time() - start_time)))

print("Brute Force")
start_time = time.time()
print("part1:", math.prod(win_bf(t,d) for t,d in races), "part2:", win_bf(t,d))
print("--- %s seconds ---" % ((time.time() - start_time)))


Root calculation
part1: 2756160 part2: 34788142
--- 0.0010051727294921875 seconds ---
Brute Force
part1: 2756160 part2: 34788142
--- 72.09413647651672 seconds ---


In [45]:
import aocd
from collections import Counter

data = aocd.get_data(day=7, year=2023).split('\n')

def parse(s, part):
    hand, bid = s.split()
    bid,cnt = int(bid), Counter(hand)
    cards = ['23456789TJQKA', 'J23456789TQKA']
    
    if part == 2: js, cnt['J'] = cnt['J'], 0
    else: js = 0

    v = list(sorted(cnt.values(), reverse=True))
    v[0] += js
    return (v,tuple(map(cards[part-1].index,hand)),bid)

for part in [1,2]:
    T = sorted(map(lambda l:parse(l,part),data))
    res = sum( i*bid for i,(_,_,bid) in enumerate(T,start=1) )
    print("Part"+str(part)+":",res)

Part1: 251058093
Part2: 249781879


In [84]:
import aocd,math,itertools

path, _, *data = aocd.get_data(day=8, year=2023).split('\n')
graph = {l[:3]:{'L':l[7:10],'R':l[12:15]} for l in data}

def solve(node,part):
    for c in itertools.count():
        node = graph[node][path[c % len(path)]]
        if node == 'ZZZ' or (part == 2 and node[-1] == 'Z'):
            return c+1

nodes = [ k for k in graph if k[-1] == 'A']

print("Part1:", solve("AAA",1), "Part2:", math.lcm(*(solve(node, 2) for node in nodes)))

Part1: 17621 Part2: 20685524831999


In [82]:
import aocd,math,itertools
path, _, *data = aocd.get_data(day=8, year=2023).split('\n')
graph = {l[:3]:(l[7:10],l[12:15]) for l in data}

def gdepth(nodes):
    for i in itertools.count(1):
        nodes = { dest for node in nodes for dest in graph[node] }
        if any(node[-1]=='Z' for node in nodes): return i

cycles = {node:gdepth({node})*len(path) for node in graph if node[-1] == 'A'}

print('Part1:', cycles['AAA'], 'Part2', math.lcm(*cycles.values()))

Part1: 17621 Part2 20685524831999


In [200]:
import aocd

data = aocd.get_data(day=9, year=2023).split('\n')
data = list(map(lambda x: list(map(int,x.split())),data))

def d(l): return [b-a for a,b in zip(l,l[1:])]
def solve(l): return l[0] - solve(d(l)) if any(l) else 0

print("Part1:", sum(solve(l[::-1]) for l in data),
      "Part2:", sum(solve(l) for l in data))

Part1: 1684566095 Part2: 1136


In [206]:
import aocd
from more_itertools import dotproduct

data = aocd.get_data(day=9, year=2023).split('\n')
data = list(map(lambda x: list(map(int,x.split())),data))
n=len(data[0])

lg_coef = [n]
for i in range(2,n+1):
      lg_coef.append(-lg_coef[-1]*(n-i+1)//i)

print("Part1:", sum(dotproduct(l[::-1],lg_coef) for l in data),
      "Part2:", sum(dotproduct(l,lg_coef) for l in data))

Part1: 1684566095 Part2: 1136


In [214]:
from itertools import product

data = '''7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ'''.split('\n')
n,m = len(data), len(data[0])

dirs = {(1,0):'E', (-1,0):'W',(0,1):'S',(0,-1):'N'}
maps = {'-':{(1,0):(1,0), (-1,0):(-1,0)},
        '|':{(0,1):(0,1), (0,-1):(0,-1)},
        'L':{(0,1):(1,0), (-1,0):(0,-1)},
        'J':{(0,1):(-1,0), (1,0):(0,-1)},
        '7':{(1,0):(0,1), (0,-1):(-1,0)},
        'F':{(0,-1):(1,0), (-1,0):(0,1)},
        '.':{}}

S = next((i,j) for i,j in product(range(m),range(n)) if data[i][j] == 'S')

def tadd(i,j,k,l): return (i+k,j+l)
def isok(i,j): return (0<=i<m and 0<=j<n)

for di in dirs:
    l = 1
    pos = tadd(*S,*di)
    while isok(*pos) and pos != S and pos != '.' and di in maps[data[pos[0]][pos[1]]]:
        di = maps[data[pos[0]][pos[1]]][di]
        pos = tadd(*pos,*di)
        l+=1
    if pos == S: print(l)



In [208]:
import aocd,math,itertools

path, _, *data = aocd.get_data(day=8, year=2023).split('\n')
graph = {l[:3]:{'L':l[7:10],'R':l[12:15]} for l in data}

def solve(node,part):
    for c in itertools.count():
        node = graph[node][path[c % len(path)]]
        if node == 'ZZZ' or (part == 2 and node[-1] == 'Z'):
            return c+1

nodes = [ k for k in graph if k[-1] == 'A']

print("Part1:", solve("AAA",1), "Part2:", math.lcm(*(solve(node, 2) for node in nodes)))

Part1: 17621 Part2: 20685524831999


In [295]:
import aocd
from itertools import count

data = aocd.get_data(day=10, year=2023).split('\n')
m,n = len(data), len(data[0])

dirs = [(1,0),(-1,0),(0,1),(0,-1)]
maps = {'-':{(1,0):(1,0), (-1,0):(-1,0)},
        '|':{(0,1):(0,1), (0,-1):(0,-1)},
        'L':{(0,1):(1,0), (-1,0):(0,-1)},
        'J':{(0,1):(-1,0), (1,0):(0,-1)},
        '7':{(1,0):(0,1), (0,-1):(-1,0)},
        'F':{(0,-1):(1,0), (-1,0):(0,1)},
        '.':{}}

data = {(i,j):c for j,line in enumerate(data) for i,c in enumerate(line)}
S = next(pos for pos in data if data[pos] == 'S')

def tadd(a,b): return (a[0]+b[0],a[1]+b[1])

for di in dirs:
    path, di0, pos = {S}, di, S
    for l in count():
        pos = tadd(pos,di)
        if pos in data and data[pos] in '-|LJ7F' and di in maps[data[pos]]:
            di = maps[data[pos]][di]
            path.add(pos)
        else: break
    if pos == S:
        data[S] = next(l for l in maps if di in maps[l] and maps[l][di] == di0)
        break

part2, in_dir = 0 # 0=out, 1=in left or below, 2=in right
for i in range(m):
    in_dir_map=[[1,2,1,0,0],[0,1,2,1,0],[0,0,0,0,1]]
    for j in range(n):
        if (i,j) not in path and in_dir: part2+= 1
        elif (i,j) in path and data[(i,j)] in '-F7LJ':
            out_map = in_dir_map[in_dir]['-F7LJ'.index(data[(i,j)])]

print("Part1:", len(path)//2, "Part2:", part2)

Part1: 6599 Part2: 477


In [355]:
import aocd
from itertools import product

def offset(pos,di): return (pos[0]+dirs[di][0],pos[1]+dirs[di][1])

dirs = [(1,0),(0,1),(-1,0),(0,-1)]
letters = "-F7LJ|S."
maps = [{0,2},{0,1},{1,2},{0,3},{2,3},{1,3},{0,1,2,3},{}]

data = aocd.get_data(day=10, year=2023).split('\n')
m,n = len(data), len(data[0])
s = letters.index('S')

sparse = {(i,j):letters.index(data[j][i]) for i,j in product(range(n),range(m))}
neighbours = {pos:{offset(pos, di) for di in maps[idx]} for pos, idx in sparse.items()}
S = next(pos for pos in sparse if sparse[pos] == s)

for next_s in neighbours[S]:
    pos, path, next_pos = S, {S}, next_s
    while next_pos in sparse and pos in neighbours[next_pos] and sparse[next_pos] < 6:
        path.add(next_pos)
        pos, next_pos = next_pos, next(p for p in neighbours[next_pos] if p!=pos)
    if next_pos == S:
        sparse[S] = next( k for k in range(1,len(maps))
                       if {next_s, pos} == {offset(S,di) for di in maps[k]})
        break

part2, in_dir = 0, 0 # 0=out, 1=in left or below, 2=in right
for i in range(m):
    in_dir_map=[[1,2,1,0,0,0,0,0],[0,1,2,1,0,1,1,1],[0,0,0,0,1,2,2,2]]
    for j in range(n):
        if (i,j) in path: in_dir = in_dir_map[in_dir][sparse[(i,j)]]
        else: part2+= (in_dir>0)

print("Part1:", len(path)//2, "Part2:", part2)

Part1: 6599 Part2: 477


SyntaxError: invalid syntax (1206914287.py, line 1)